<a href="https://colab.research.google.com/github/AnnaIsoyan/dl_project/blob/main/change_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls

In [ ]:
cd 'drive/MyDrive/Change_detection/Data'

In [ ]:
!unzip train.zip -d 'train'
!unzip test.zip -d 'test'
!unzip val.zip  -d 'val'

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import keras
from keras.layers import Conv2D, Conv2DTranspose, InputLayer, Layer, Input, Dropout, MaxPool2D, concatenate, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from math import log
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import metrics

In [ ]:
def read_img(path):
  img_l = []
  images = sorted(
      [
          os.path.join(path, fname)
          for fname in os.listdir(path)
          if fname.endswith(".png")
      ]
  )
  for i in images:
    img = cv2.imread(i)
    # img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img,(256,256))/256
    img_l.append(img)

  return img_l


In [ ]:
train_A = read_img('train/A')
train_B = read_img('train/B')
train_label = read_img('train/label')

In [ ]:
val_A = read_img('val/A')
val_B = read_img('val/B')
val_label = read_img('val/label')

In [ ]:
train_A = np.array(train_A)
train_B = np.array(train_B)
train_label = np.array(train_label)
val_A = np.array(val_A)
val_B = np.array(val_B)
val_label = np.array(val_label)

In [ ]:
train_A.shape

In [ ]:
plt.imshow(train_A[0])
print(train_A[5].shape)

In [ ]:
plt.imshow(train_B[0]);

In [ ]:
plt.imshow(train_label[0]);

In [ ]:
class SUD_enc(Layer):
    """SiamUnet_diff segmentation network."""

    def __init__(self, filters, rate, pooling=True):
      super(SUD_enc, self).__init__()
      self.pooling = pooling
      self.filters = filters
      self.rate = rate
      self.pool = MaxPool2D(pool_size=(2,2))

      self.c1 = Conv2D(self.filters, kernel_size=3, padding='same', activation='relu')
      self.c11 = Conv2D(16, kernel_size=3, padding='same', activation='relu')
      self.c111 = Conv2D(16, kernel_size=3, padding='same', activation='relu')
      self.bn1 = tf.keras.layers.BatchNormalization(momentum=0.99,epsilon=0.001)
      self.drop1 = Dropout(self.rate)

      self.c2 = Conv2D(16, kernel_size=3, padding='same', activation='relu')
      self.c22 = Conv2D(32, kernel_size=3, padding='same', activation='relu')
      self.c222 = Conv2D(32, kernel_size=3, padding='same', activation='relu')
      self.bn2 = tf.keras.layers.BatchNormalization(momentum=0.99,epsilon=0.001)
      self.drop2 = Dropout(self.rate)

      self.c3 = Conv2D(32, kernel_size=3, padding='same', activation='relu')
      self.c33 = Conv2D(64, kernel_size=3, padding='same', activation='relu')
      self.c333 = Conv2D(64, kernel_size=3, padding='same', activation='relu')
      self.c3333 = Conv2D(64, kernel_size=3, padding='same', activation='relu')
      self.bn3 = tf.keras.layers.BatchNormalization(momentum=0.99,epsilon=0.001)
      self.drop3 = Dropout(self.rate)

      self.c4 = Conv2D(64, kernel_size=3, padding='same', activation='relu')
      self.c44 = Conv2D(128, kernel_size=3, padding='same', activation='relu')
      self.c444 = Conv2D(128, kernel_size=3, padding='same', activation='relu')
      self.c4444 = Conv2D(128, kernel_size=3, padding='same', activation='relu')
      self.bn4 = tf.keras.layers.BatchNormalization(momentum=0.99,epsilon=0.001)
      self.drop4 = Dropout(self.rate)

      self.ct1 = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same')

      self.cv1 = Conv2DTranspose(128, kernel_size=3, padding='same', activation='relu')
      self.cv11 = Conv2DTranspose(128, kernel_size=3, padding='same', activation='relu')
      self.cv111 = Conv2DTranspose(64, kernel_size=3, padding='same', activation='relu')
      self.dr1 = Dropout(self.rate)
      
      self.ct2 = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same')

      self.cv2 = Conv2DTranspose(64, kernel_size=3, padding='same', activation='relu')
      self.cv22 = Conv2DTranspose(64, kernel_size=3, padding='same', activation='relu')
      self.cv222 = Conv2DTranspose(32, kernel_size=3, padding='same', activation='relu')

      self.dr2 = Dropout(self.rate)

      self.ct3 = Conv2DTranspose(32, kernel_size=3, strides=2, padding='same')

      self.cv3 = Conv2DTranspose(32, kernel_size=3, padding='same', activation='relu')
      self.cv33 = Conv2DTranspose(16, kernel_size=3, padding='same', activation='relu')
      self.dr3 = Dropout(self.rate)

      self.ct4 = Conv2DTranspose(16, kernel_size=3, strides=2, padding='same')

      self.cv4 = Conv2DTranspose(16, kernel_size=3, padding='same', activation='relu')
      self.cv44 = Conv2DTranspose(3, kernel_size=3, padding='same', activation='sigmoid')
      self.dr4 = Dropout(self.rate)


      self.pool = MaxPool2D(pool_size=(2,2))
    def call(self, x1,x2):
      # Stage 1
      x11 = self.bn1(self.drop1(self.c1(x1)))
      # x11 = self.drop1(self.c11(x11))
      x11 = self.c11(x11)
      # x11 = self.c111(x11)
      y11 = self.pool(x11)

      x21 = self.bn1(self.drop1(self.c1(x2)))
      # x21 = self.drop1(self.c11(x21))
      x21 = self.c11(x21)
      # x21 = self.c111(x21)
      y21 = self.pool(x21)

      #stage 2
      x12 = self.bn2(self.drop2(self.c2(y11)))
      # x12 = self.drop2(self.c22(x12))
      x12 = self.c22(x12)
      # x12 = self.c222(x12)
      y12 = self.pool(x12) 
       
      x22 = self.bn2(self.drop2(self.c2(y21)))
      # x22 = self.drop2(self.c22(x22))
      x22 = self.c22(x22)
      # x22 = self.c222(x22)
      y22 = self.pool(x22)       
      
      #stage 3 
      x13 = self.bn3(self.drop3(self.c3(y12)))
      # x13 = self.drop3(self.c33(x13))
      x13 = self.c33(x13)
      x13 = self.c333(x13)
      # x13 = self.c3333(x13) 
      y13 = self.pool(x13)

      x23 = self.bn3(self.drop3(self.c3(y22)))
      # x23 = self.drop3(self.c33(x23))
      x23 = self.c33(x23)
      x23 = self.c333(x23)
      # x23 = self.c3333(x23) 
      y23 = self.pool(x23)

      #stage 4 
      x14 = self.bn4(self.drop4(self.c4(y13)))
      # x14 = self.drop4(self.c44(x14))
      x14 = self.c44(x14)
      x14 = self.c444(x14)
      # x14 = self.c4444(x14)     
      y14 = self.pool(x14)

      x24 = self.bn4(self.drop4(self.c4(y23)))
      # x24 = self.drop4(self.c44(x24))
      x24 = self.c44(x24)
      x24 = self.c444(x24)
      # x24 = self.c4444(x24) 
      y24 = self.pool(x24)

      #stage 4d
      p1 = self.ct1(y14)
      p1 = self.cv1(concatenate([p1, tf.abs(x24 - x14)]))
      p1 = self.dr1(p1)
      p1 = self.cv11(p1)
      # p1 = self.dr1(p1)
      p1 = self.cv111(p1)
      # p1 = self.dr1(p1)

      #stage 3d
      p2 = self.ct2(p1)
      p2 = self.cv2(concatenate([p2, tf.abs(x23 - x13)]))
      p2 = self.dr2(p2)
      p2 = self.cv22(p2)
      # p2 = self.dr2(p2)
      p2 = self.cv222(p2)
      # p2 = self.dr2(p2) 
      
      #stage 2d
      p3 = self.ct3(p2)
      p3 = self.cv3(concatenate([p3, tf.abs(x22 - x12)]))
      # p3 = self.dr3(p3)
      p3 = self.cv33(p3)
      # p3 = self.dr3(p3)

      #stage 1 d
      p4 = self.ct4(p3)
      p4 = self.cv4(concatenate([p4, tf.abs(x21 - x11)]))
      p4 = self.dr4(p4)
      p4 = self.cv44(p4)


      return p4


In [ ]:
input_layer1 = Input(shape=(256,256,3))
input_layer2 = Input(shape=(256,256,3))

output = SUD_enc(16,0.2)(input_layer1,input_layer2)
output

In [ ]:
unetsiam_diff = Model(
    inputs=[input_layer1,input_layer2], outputs=output
)


In [ ]:
unetsiam_diff.compile(loss='binary_crossentropy',
                      optimizer=Adam(learning_rate=0.01),
                      metrics=['categorical_accuracy'])

In [ ]:
unetsiam_diff.summary()

In [ ]:
callbacks =[
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("SiamUNet_diff.h5", save_best_only=True),
]

In [ ]:
unetsiam_diff.fit(
    x = [train_A,train_B],
    y = train_label,
    validation_data=([val_A,val_B], val_label),
    epochs=50,
    batch_size=32,
    # callbacks=[metrics],
)


In [ ]:
# X_train = train_A
# X2_train = train_B
# y_train = train_label
# for i in range(90):
#   X_train[i] = train_A[0]
#   X2_train[i] = train_B[0]
#   y_train[i] = train_label[0] 
# for i in range(90,180):
#   X_train[i] = train_A[1]
#   X2_train[i] = train_B[1]
#   y_train[i] = train_label[1]
# for i in range(180,270):
#   X_train[i] = train_A[2]
#   X2_train[i] = train_B[2]
#   y_train[i] = train_label[2]  
# for i in range(270,360):
#   X_train[i] = train_A[3]
#   X2_train[i] = train_B[3]
#   y_train[i] = train_label[3]
# for i in range(360,445):
#   X_train[i] = train_A[4]
#   X2_train[i] = train_B[4]
#   y_train[i] = train_label[4]

In [ ]:
# unetsiam_diff.fit(
#     x = [X_train,X2_train],
#     y = y_train,
#     validation_data=([val_A,val_B], val_label),
#     epochs=10,
#     batch_size=32,
#     # callbacks=[metrics],
# )


In [ ]:
unetsiam_diff.save_weights(filepath='weight.h5')
unetsiam_diff.save('model')

# model = keras.models.load_model('path/to/location')

In [ ]:
test_A = read_img('test/A')
test_B = read_img('test/B')
test_label = read_img('test/label')

In [ ]:
test_A = np.array(test_A)
test_B = np.array(test_B)
test_label = np.array(test_label)

In [ ]:
plt.imshow(val_A[63]);

In [ ]:
plt.imshow(val_B[63]);

In [ ]:
plt.imshow(val_label[63]);

In [ ]:
test_B[0].shape

In [ ]:
cd '/content'

In [ ]:
cd 'drive/MyDrive/My_drive/Change_detection/Data'

In [ ]:
# unetsiam_diff.load_weights("final_weight.h5")

In [ ]:
pred=unetsiam_diff.predict(x=[test_A,test_B],
                           verbose="auto")

In [ ]:
pred1=unetsiam_diff.predict(x=[val_A,val_B])

In [ ]:
# unetsiam_diff.get_weights()

In [ ]:
plt.imshow(val_A[9])

In [ ]:
plt.imshow(pred[9]);

In [ ]:
plt.imshow(val_label[9]);